In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from nltk.tokenize import sent_tokenize
from keras import layers, models, optimizers
import csv



In [2]:
import pandas as pd
import numpy as np

In [3]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import random

[nltk_data] Downloading package punkt to /Users/james/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
csv.field_size_limit(999999)
train = pd.read_csv('raw_data/fulltrain.csv', header = None, names=['class','text'])
test = pd.read_csv("raw_data/balancedtest.csv", header = None, names=['class','text'])

In [5]:
train['class'].value_counts()

class
3    17870
1    14047
4     9995
2     6942
Name: count, dtype: int64

In [6]:
one = train.loc[train['class'] == 1]
two = train.loc[train['class'] == 2]
three = train.loc[train['class'] == 3]
four = train.loc[train['class'] == 4]

In [7]:
train = pd.concat([train, one.sample(n=3953), two.sample(n=6000), two.sample(n=5058), three.sample(n=130), four.sample(n=8005)], ignore_index=True)


In [8]:
train['class'].value_counts()

class
1    18000
2    18000
3    18000
4    18000
Name: count, dtype: int64

In [9]:
x_train = train['text']
x_test = test['text']

In [10]:
tokenizer = Tokenizer(num_words=700)
tokenizer.fit_on_texts(x_train.values)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(x_train)
test_sequences = tokenizer.texts_to_sequences(x_test)

padding_type = 'post'
truncating_type = 'post'

In [11]:
vocab_size = len(word_index) + 1
max_length = 1000

X_train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=truncating_type)
X_test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=truncating_type)


In [12]:
vocab_size = len(word_index) + 1                          

maxlen = 1000

X_train = pad_sequences(train_sequences, padding='post', maxlen=maxlen)
X_test = pad_sequences(test_sequences, padding='post', maxlen=maxlen)

In [13]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from tensorflow.keras.layers import Embedding

from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate




In [14]:
def create_embedding_matrix(word_vectors, word_index, embedding_dim):
    vocab_size = len(word_index) + 1 
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, idx in word_index.items():
        if word in word_vectors:
            embedding_matrix[idx] = word_vectors[word]
    return embedding_matrix

word2vec_path = 'word2vec-google-news-300.model' 
word2vec = KeyedVectors.load(word2vec_path)

embedding_dim = 300  
embedding_matrix = create_embedding_matrix(word2vec, tokenizer.word_index, embedding_dim)



In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
tfidf_vectorizer = TfidfVectorizer(max_features=10000)  
tfidf_matrix = tfidf_vectorizer.fit_transform(x_train) 
tfidf_feature_length = tfidf_matrix.shape[1] 

In [17]:
y_train = train['class'] - 1
y_test = test['class'] - 1

In [18]:
num_classes = 4
y_train_categorical = to_categorical(y_train, num_classes=num_classes)
y_test_categorical = to_categorical(y_test, num_classes=num_classes)

In [19]:
y_test_categorical

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]])

In [20]:
sequence_input = Input(shape=(max_length,), dtype='int32')
tfidf_input = Input(shape=(tfidf_feature_length,))

In [21]:
import tensorflow as tf
embedding_layer = Embedding(input_dim=embedding_matrix.shape[0],  
                            output_dim=embedding_matrix.shape[1],  
                            embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),  
                            input_length=max_length, 
                            trainable=False)(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedding_layer)

x = Conv1D(64, 5, activation='relu')(x)  

x = GlobalMaxPooling1D()(x)

x = Dense(64, activation='relu')(x)
cnn_path = Dropout(0.5)(x)


/Users/james/Documents/NUS/Y2S2/CS4248/Labeled-Unreliable-News/venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [22]:
y = Dense(64, activation='relu')(tfidf_input) 
tfidf_path = Dropout(0.5)(y)


In [23]:
from tensorflow.keras.layers import Concatenate

combined = Concatenate()([cnn_path, tfidf_path])

In [24]:
output = Dense(4, activation='softmax')(combined)

model = Model(inputs=[sequence_input, tfidf_input], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-3), metrics=['accuracy'])


In [25]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 1000)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 1000, 300) │ 77,786,100 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 996, 128)  │    192,128 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 992, 64)   │     41,024 │ conv1d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 64)        │          0 │ conv1d_1[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 10000)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      4,160 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │    640,064 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64)        │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 128)       │          0 │ dropout[0][0],    │
│ (Concatenate)       │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 4)         │        516 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 78,663,992 (300.08 MB)

 Trainable params: 877,892 (3.35 MB)

 Non-trainable params: 77,786,100 (296.73 MB)

In [26]:
X_train_sequences = tokenizer.texts_to_sequences(x_train) 
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length)
X_train_tfidf = tfidf_vectorizer.transform(x_train).toarray() 


In [27]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min', restore_best_weights=True)


In [28]:
X_test_sequences = tokenizer.texts_to_sequences(x_test) 
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length)
X_test_tfidf = tfidf_vectorizer.transform(x_test).toarray() 

In [29]:
model1 = model

In [30]:
model.fit([X_train_padded, X_train_tfidf], y_train_categorical, batch_size=128, epochs=30, validation_data=([X_test_padded, X_test_tfidf], y_test_categorical), callbacks=[early_stopping])


Epoch 1/30
563/563 ━━━━━━━━━━━━━━━━━━━━ 224s 396ms/step - accuracy: 0.8071 - loss: 0.5106 - val_accuracy: 0.6437 - val_loss: 0.9982
Epoch 2/30
563/563 ━━━━━━━━━━━━━━━━━━━━ 219s 389ms/step - accuracy: 0.9842 - loss: 0.0577 - val_accuracy: 0.6877 - val_loss: 0.9612
Epoch 3/30
563/563 ━━━━━━━━━━━━━━━━━━━━ 221s 392ms/step - accuracy: 0.9938 - loss: 0.0261 - val_accuracy: 0.6877 - val_loss: 1.0714
Epoch 4/30
563/563 ━━━━━━━━━━━━━━━━━━━━ 221s 393ms/step - accuracy: 0.9971 - loss: 0.0146 - val_accuracy: 0.6610 - val_loss: 1.3932
Epoch 5/30
563/563 ━━━━━━━━━━━━━━━━━━━━ 220s 391ms/step - accuracy: 0.9979 - loss: 0.0088 - val_accuracy: 0.6807 - val_loss: 1.3629
Epoch 6/30
563/563 ━━━━━━━━━━━━━━━━━━━━ 224s 398ms/step - accuracy: 0.9988 - loss: 0.0063 - val_accuracy: 0.6377 - val_loss: 1.7966
Epoch 7/30
563/563 ━━━━━━━━━━━━━━━━━━━━ 229s 406ms/step - accuracy: 0.9992 - loss: 0.0049 - val_accuracy: 0.6597 - val_loss: 1.5463
Epoch 7: early stopping
Restoring model weights from the end of the best epo

In [31]:
model1.fit([X_train_padded, X_train_tfidf], y_train_categorical, batch_size=128, epochs=30, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 215s 423ms/step - accuracy: 0.9922 - loss: 0.0299 - val_accuracy: 0.9981 - val_loss: 0.0135
Epoch 2/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 214s 421ms/step - accuracy: 0.9970 - loss: 0.0120 - val_accuracy: 0.9993 - val_loss: 0.0080
Epoch 3/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 213s 421ms/step - accuracy: 0.9981 - loss: 0.0079 - val_accuracy: 0.9999 - val_loss: 0.0029
Epoch 4/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 211s 417ms/step - accuracy: 0.9990 - loss: 0.0052 - val_accuracy: 0.9999 - val_loss: 0.0018
Epoch 5/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 210s 415ms/step - accuracy: 0.9992 - loss: 0.0039 - val_accuracy: 0.9996 - val_loss: 0.0037
Epoch 6/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 212s 419ms/step - accuracy: 0.9995 - loss: 0.0031 - val_accuracy: 0.9999 - val_loss: 9.4380e-04
Epoch 7/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 213s 420ms/step - accuracy: 0.9994 - loss: 0.0028 - val_accuracy: 0.9999 - val_loss: 9.3976e-04
Epoch 8/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 211s 417ms/step - accuracy: 

In [32]:
y_pred = model.predict([X_test_padded, X_test_tfidf])
y_pred_classes = y_pred.argmax(axis=1)

94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step


In [33]:
y_pred1 = model1.predict([X_test_padded, X_test_tfidf])
y_pred1_classes = y_pred1.argmax(axis=1)

94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step


In [34]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [35]:
score = f1_score(y_test, y_pred_classes, average='macro')
print('Score on validation = {}'.format(score))

Score on validation = 0.6887103321310872


In [36]:
print('Classification report on test data:')
print(classification_report(y_test, y_pred_classes))

Classification report on test data:
              precision    recall  f1-score   support

           0       0.86      0.68      0.76       750
           1       0.72      0.49      0.58       750
           2       0.58      0.66      0.62       750
           3       0.68      0.95      0.80       750

    accuracy                           0.70      3000
   macro avg       0.71      0.70      0.69      3000
weighted avg       0.71      0.70      0.69      3000



In [37]:
score = f1_score(y_test, y_pred1_classes, average='macro')
print('Score on validation = {}'.format(score))

Score on validation = 0.6887103321310872


In [38]:
print('Classification report on test data:')
print(classification_report(y_test, y_pred1_classes))

Classification report on test data:
              precision    recall  f1-score   support

           0       0.86      0.68      0.76       750
           1       0.72      0.49      0.58       750
           2       0.58      0.66      0.62       750
           3       0.68      0.95      0.80       750

    accuracy                           0.70      3000
   macro avg       0.71      0.70      0.69      3000
weighted avg       0.71      0.70      0.69      3000

